In [ ]:
# '''''''''''''''''
# Author: Zijie Luo
# ID: a1866762
# '''''''''''''''''

In [3]:
# Import Necessary Library
import os
import pandas as pd
import numpy as np
import cvxpy as cp

In [13]:
# Process train and test dataset

data_train, data_test = pd.read_csv("train.csv",header=None), pd.read_csv("test.csv",header=None)

# Train dataset

## extract label and features
train_label, train_feature = data_train.iloc[:,0], data_train.iloc[:,1:]

X_train, y_train_label = train_feature.iloc[:4000,:].to_numpy(), train_label.iloc[:4000].to_numpy()
X_val, y_val_label = train_feature.iloc[4000:,:].to_numpy(), train_label.iloc[4000:].to_numpy()

# print(X_train.shape) # numpy np.shape->Capture the dimension

# Test dataset
test_label, test_feature = data_test.iloc[:,0].to_numpy(), data_test.iloc[:,1:].to_numpy()

In [ ]:
# Question2
''' 
implement the training and testing algorithms of 
soft-margin Linear Support Vector Machine from 
its primal form
'''

## Training
def svm_train_primal(data_train, label_train, regularisation_para_C):
    n_samples, n_features = data_train.shape # [4000,200] -> n_samples=4000, n_features=200
    # w 应该代表特征，该数据集每个sample有200个feature，总共有4000个samples被用来训练模型
    w = cp.Variable(n_features) # w should be find through dataset, it means how many samples should in there
    b = cp.Variable()
    zeta_i = cp.Variable(n_samples)

    # svm classify the feature into (-1,1) but not (0,1), so we re-encode the label
    label_train = np.where(label_train == 0, -1, label_train)

    y_i = label_train
    decision = data_train @ w + b
    
    objective = cp.Minimize(0.5 * cp.norm(w, 2) + regularisation_para_C * cp.sum(zeta_i))
    constraints = [cp.multiply(y_i, decision) >= 1 - zeta_i, zeta_i >= 0]
    
    problem = cp.Problem(objective, constraints)
    problem.solve()
    # Check the status of the solution, ensure the solver can find optimal solution
    if problem.status == 'optimal':
        print("Solution found is optimal.")
    else:
        print(f"Problem not solved optimally: {problem.status}")
    
    svm_model = {'w': w.value, 'b': b.value, 'z': zeta_i.value}
    return svm_model


In [ ]:
## Test accuracy
def svm_predict_primal(data_test, label_test, svm_model):
    w = svm_model['w']
    b = svm_model['b']
    
    predictions = data_test @ w + b
    predictions[predictions >= 0] = 1
    predictions[predictions < 0] = -1
    
    test_accuracy = np.mean(predictions == label_test)
    return test_accuracy


In [ ]:
## Apply Model

# define regularization C range to find the optimal value, [1] define baseline
regularization_para_C = [1] + list(range(0, 31, 5))

# Start training the SVM
for C in regularization_para_C:
    svm_model = svm_train_primal(X_train, y_train_label, C)
    test_accuracy = svm_predict_primal(X_val, y_val_label, svm_model)
    print(f"Val accuracy with C={C}: {test_accuracy}")

# # Print the solution for b and the sum of all dimensions of w
# print("Solution for b:", svm_model['b'])
# print("Sum of all dimensions of w:", np.sum(svm_model['w']))
# print("solution for zeta_i", np.sum(svm_model['z']))
